# Dask Delayed

Материалы:
* Макрушин С.В. Лекция 13: Dask Delayed
* https://docs.dask.org/en/latest/delayed.html
* Jesse C. Daniel. Data Science with Python and Dask.


## Задачи для совместного разбора

![](https://i.imgur.com/AwiN8y6.png)
![](https://i.imgur.com/ceY6guU.png)

1. Напишите 2 функции, имитирующие CPU-bound задачу и IO-bound задачу:

`cpu_task()`: генерирует 100 тыс. случайных чисел и возвращает их сумму (без использования `numpy`)

`io_task()`: "спит" 0.1 сек, затем генерирует случайное число и возвращает его

Замерьте время выполнения 100 последовательных вызовов каждой из этих функций. Распараллелив вычисления при помощи `dask.delayed`, сократите время выполнения. Исследуйте, как зависит время вычислений от выбранного планировщика `scheduler`.

In [1]:
import random
import time


def cpu_task():
	lst = [random.randint(0, 10) for _ in range(100_000)]
	return sum(lst)

def io_task():
	time.sleep(0.1)
	return random.randint(0, 10)

In [2]:
%%time
res = [cpu_task() for _ in range(100)]

Wall time: 4.07 s


In [3]:
%%time
res = [io_task() for _ in range(100)]

Wall time: 10.8 s


In [8]:
from dask import delayed
from dask import compute

In [5]:
cpu_task_delayed = delayed(cpu_task)

In [6]:
cpu_task_delayed

Delayed('cpu_task-ac40120a-5ea1-41a4-a72f-fcc99c234c17')

In [9]:
%%time
res = [cpu_task_delayed() for _ in range(100)]
compute(res)

Wall time: 4.11 s


([499472,
  499501,
  500654,
  499008,
  499409,
  501652,
  499998,
  500428,
  500694,
  500807,
  500337,
  500528,
  498999,
  500009,
  500968,
  500769,
  499749,
  500257,
  499002,
  500213,
  499322,
  499651,
  499006,
  499855,
  500670,
  499468,
  500032,
  500305,
  500738,
  500681,
  499722,
  499936,
  499420,
  499469,
  499147,
  498455,
  499582,
  500873,
  498704,
  499185,
  499081,
  499935,
  501697,
  499051,
  500626,
  498240,
  499822,
  500641,
  500555,
  499455,
  499789,
  500923,
  499739,
  500361,
  499573,
  500466,
  499965,
  499165,
  500741,
  498951,
  498171,
  501325,
  501753,
  500672,
  499119,
  497931,
  499771,
  500095,
  498941,
  499373,
  501018,
  500978,
  500876,
  499428,
  501467,
  500921,
  500299,
  498792,
  500449,
  502080,
  500052,
  500557,
  500347,
  498825,
  502218,
  500087,
  500189,
  499657,
  499366,
  500435,
  499825,
  500746,
  499879,
  499346,
  502463,
  501392,
  500078,
  499590,
  500666,
  499117],

In [10]:
%%time
res = [cpu_task_delayed() for _ in range(100)]
res_computed = compute(res, scheduler='multiprocessing')

Wall time: 1.17 s


In [11]:
io_task_delayed = delayed(io_task)

In [13]:
%%time
res = [io_task_delayed() for _ in range(100)]
res_computed = compute(res, scheduler='threading')

Wall time: 756 ms


In [14]:
%%time
res = [io_task_delayed() for _ in range(100)]
res_computed = compute(res, scheduler='multiprocessing')

Wall time: 1.44 s


## Лабораторная работа 14

1. Напишите функцию, которая считывает файл формата xml из каталога `reviewers_full` и по данным этого файла формирует список словарей, содержащих следующие ключи: `id`, `username`, `name`, `sex`, `country`, `mail`, `registered`, `birthdate`, `name_prefix`, `country_code`. Часть из этих значений в исходном файле хранится в виде тэгов, часть - в виде атрибутов тэгов. Для конкретного человека какие-то из этих ключей могут отсутствовать. 



In [59]:
from bs4 import BeautifulSoup as bs

# soup = bs(xml_text, "xml")
user_list = []

content = []
with open("14_delayed_data/reviewers_full/reviewers_full_0.xml", "r") as f:
	users = bs(f, "lxml")
	rows = users.find_all("user")
	c = 0
	for i in rows:
		# try:
		user_dict = {
			"id": i.find("id"),
			"username": i.find("username"),
			"name": i.find("name"),
			"sex": i.find("sex"),
			"country": i.find("country"),
			"mail": i.find("mail"),
			"registered": i.find("registered"),
			"birthdate": i.find("birthdate"),
			"name_prefix": i.find("user"),
			"country_code": i.find("country")
		}
		# except Exception as e:
		# 	e
		user_list.append(user_dict)
		c += 1
		if c == 10:
			break
user_list

[{'id': <id>556011</id>,
  'username': <username>gabrielacalhoun</username>,
  'name': None,
  'sex': <sex>F</sex>,
  'country': None,
  'mail': None,
  'registered': None,
  'birthdate': <birthdate>1988-01-25</birthdate>,
  'name_prefix': None,
  'country_code': None},
 {'id': <id>1251087</id>,
  'username': <username>qbaxter</username>,
  'name': None,
  'sex': None,
  'country': <country code="NO">Norway</country>,
  'mail': <mail>qware@gmail.com</mail>,
  'registered': None,
  'birthdate': <birthdate>1985-01-19</birthdate>,
  'name_prefix': None,
  'country_code': <country code="NO">Norway</country>},
 {'id': <id>537188</id>,
  'username': <username>crosschristopher</username>,
  'name': <name>Dana Moore</name>,
  'sex': None,
  'country': None,
  'mail': <mail>stephaniestrong@yahoo.com</mail>,
  'registered': <registered>2018-11-21</registered>,
  'birthdate': <birthdate>1955-07-03</birthdate>,
  'name_prefix': None,
  'country_code': None},
 {'id': <id>250427</id>,
  'username': 

In [139]:
import os
import xml.etree.ElementTree as ET
from dask import delayed as dd
from dask import bag as db
from dask import dataframe as df
from dask import compute
from pprint import pp

In [119]:
def xml_to_dict(file):
	tree = ET.parse(file)
	root = tree.getroot()
	user_list = []
	for child in root.findall('user'):
		user_dict = {
				"id": None,
				"username": None,
				"name": None,
				"sex": None,
				"country": None,
				"mail": None,
				"registered": None,
				"birthdate": None,
				"name_prefix": None,
				"country_code": None,
			}

		try:
			user_dict["id"] = child.find("id").text
		except:
			pass

		try:
			user_dict["username"] = child.find("username").text
		except:
			pass

		try:
			user_dict["name"] = child.find("name").text
		except:
			pass

		try:
			user_dict["sex"] = child.find("sex").text
		except:
			pass

		try:
			user_dict["country"] = child.find("country").text
		except:
			pass

		try:
			user_dict["mail"] = child.find("mail").text
		except:
			pass

		try:
			user_dict["registered"] = child.find("registered").text
		except:
			pass

		try:
			user_dict["birthdate"] = child.find("birthdate").text
		except:
			pass

		try:
			user_dict["name_prefix"] = child.attrib["prefix"]
		except:
			pass

		try:
			user_dict["country_code"] = child.find("country").attrib["code"]
		except:
			pass
		user_list.append(user_dict)
	return user_list

In [120]:
file = "14_delayed_data/reviewers_full/reviewers_full_0.xml"
ul = xml_to_dict(file)
ul[0]

{'id': '556011',
 'username': 'gabrielacalhoun',
 'name': None,
 'sex': 'F',
 'country': None,
 'mail': None,
 'registered': None,
 'birthdate': '1988-01-25',
 'name_prefix': 'Mrs.',
 'country_code': None}

2. Измерьте время выполнения функции из задания 1 на всех файлах из каталога `reviewers_full`. Ускорьте время выполнения, используя `dask.delayed`.

In [134]:
%%time

for filename in os.listdir("14_delayed_data/reviewers_full/"):
	with open("14_delayed_data/reviewers_full/" + filename, "r") as file:
		out = xml_to_dict(file)
		print(out[0])

{'id': '556011', 'username': 'gabrielacalhoun', 'name': None, 'sex': 'F', 'country': None, 'mail': None, 'registered': None, 'birthdate': '1988-01-25', 'name_prefix': 'Mrs.', 'country_code': None}
{'id': '394270', 'username': 'bridgesdennis', 'name': 'Melissa Vaughn', 'sex': 'F', 'country': None, 'mail': 'carmengonzales@hotmail.com', 'registered': None, 'birthdate': '1992-07-28', 'name_prefix': 'Mrs.', 'country_code': None}
{'id': '2951090', 'username': 'jesse68', 'name': None, 'sex': None, 'country': 'Jamaica', 'mail': 'kaitlyn40@yahoo.com', 'registered': '2019-03-31', 'birthdate': '1980-04-30', 'name_prefix': None, 'country_code': 'JM'}
{'id': '46542', 'username': 'dawn16', 'name': None, 'sex': None, 'country': 'Serbia', 'mail': 'murrayshannon@gmail.com', 'registered': '2019-11-24', 'birthdate': '2018-06-10', 'name_prefix': None, 'country_code': 'RS'}
{'id': '2214951', 'username': 'nancyharrison', 'name': None, 'sex': 'M', 'country': 'Mali', 'mail': None, 'registered': '2017-05-24', 

In [135]:
%%time
for i in range(len(os.listdir("14_delayed_data/reviewers_full/"))):
	out = xml_to_dict("14_delayed_data/reviewers_full/reviewers_full_"+str(i)+".xml")
	print(out[0])

{'id': '556011', 'username': 'gabrielacalhoun', 'name': None, 'sex': 'F', 'country': None, 'mail': None, 'registered': None, 'birthdate': '1988-01-25', 'name_prefix': 'Mrs.', 'country_code': None}
{'id': '394270', 'username': 'bridgesdennis', 'name': 'Melissa Vaughn', 'sex': 'F', 'country': None, 'mail': 'carmengonzales@hotmail.com', 'registered': None, 'birthdate': '1992-07-28', 'name_prefix': 'Mrs.', 'country_code': None}
{'id': '2951090', 'username': 'jesse68', 'name': None, 'sex': None, 'country': 'Jamaica', 'mail': 'kaitlyn40@yahoo.com', 'registered': '2019-03-31', 'birthdate': '1980-04-30', 'name_prefix': None, 'country_code': 'JM'}
{'id': '46542', 'username': 'dawn16', 'name': None, 'sex': None, 'country': 'Serbia', 'mail': 'murrayshannon@gmail.com', 'registered': '2019-11-24', 'birthdate': '2018-06-10', 'name_prefix': None, 'country_code': 'RS'}
{'id': '2214951', 'username': 'nancyharrison', 'name': None, 'sex': 'M', 'country': 'Mali', 'mail': None, 'registered': '2017-05-24', 

3. Задекорируйте функцию из задания 1 при помощи `dask.delayed` и создайте список `reviewers`, состоящий из 5 объектов `delayed` (по одному объекту на файл). Из списка объектов `delayed`, создайте `dask.bag` при помощи метода `db.from_delayed`. Добавьте ключ `birth_year`, в котором хранится год рождения человека. Оставьте в выборке только тех людей, которые __наверняка__ моложе 1980 года. Преобразуйте поле `id` к целому типу.

In [124]:
delayed(xml_to_dict)

Delayed('xml_to_dict-1d04e7c6-2a88-49fd-91de-063554cbb82e')

In [138]:
reviewers = []
for filename in os.listdir("14_delayed_data/reviewers_full/"):
	with open("14_delayed_data/reviewers_full/" + filename, "r") as file:
		reviewers.append(dd(xml_to_dict))
reviewers

[Delayed('xml_to_dict-4512cadf-8909-41e8-b04a-02934dd19831'),
 Delayed('xml_to_dict-65868e3e-7bf2-4af8-93b1-84164a0ee2b8'),
 Delayed('xml_to_dict-291d8264-f905-4bcf-a7a8-0bce55a76aff'),
 Delayed('xml_to_dict-9214ba40-3d49-45b4-afe0-1cc5c549dde5'),
 Delayed('xml_to_dict-2cbe8c49-f3fe-4890-9ca2-0d6113d3b025')]

In [148]:
bag = db.from_delayed(reviewers)
bag

dask.bag<bag-from-delayed, npartitions=5>

4. Из `dask.bag`, полученного в задании 3, создайте `dask.dataframe` при помощи метода `bag.to_dataframe`. Укажите столбец `id` в качестве индекса.

In [149]:
ddf = bag.to_dataframe()
ddf

TypeError: 'function' object is not iterable

Traceback
---------
  File "f:\programming\python\bigdatapt-bdpt-\venv39\lib\site-packages\dask\local.py", line 220, in execute_task
    result = _execute_task(task, data)
  File "f:\programming\python\bigdatapt-bdpt-\venv39\lib\site-packages\dask\core.py", line 119, in _execute_task
    return func(*(_execute_task(a, cache) for a in args))
  File "f:\programming\python\bigdatapt-bdpt-\venv39\lib\site-packages\dask\bag\core.py", line 2457, in safe_take
    r = list(take(n, b))
  File "f:\programming\python\bigdatapt-bdpt-\venv39\lib\site-packages\toolz\itertoolz.py", line 329, in take
    return itertools.islice(seq, n)


5. Назовем отзыв негативным, если оценка равна 0, 1 или 2. Загрузите данные о негативных отзывах из файлов архива `reviews_full` (__ЛР12__) в виде `dask.DataFrame`. Посчитайте количество отзывов с группировкой по пользователю, оставившему отзыв. Объедините результат с таблицей, полученной в задаче 4.

#### [версия 2]
* Уточнена формулировка задачи 1